#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [5]:
import graphlab






#Read some product review data

Loading reviews for a set of baby products. 

In [6]:
products = graphlab.SFrame('amazon_baby.gl/')

#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [7]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [8]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [9]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


In [10]:
graphlab.canvas.set_target('ipynb')

In [11]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [12]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [13]:
len(giraffe_reviews)

785

In [14]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [15]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [16]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [17]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [18]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


##Let's train the sentiment classifier

In [19]:
train_data,test_data = products.random_split(.8, seed=0)

In [20]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 3.155318     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 5.236003     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 6.041692     | 0.92

#Evaluate the sentiment model

In [21]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.222493887531 | 0.00424454273077 | 28036 | 5317 |
 | 0.0010000000475  | 0.777506112469 |  0.995755457269  | 28036 | 5317 |
 | 0.00200000009499 | 0.738386308068 |  0.994471393922  | 28036 | 5317 |
 | 0.00300000002608 | 0.715629114162 |  0.993829362249  | 28036 | 5317 |
 | 0.00400000018999 | 0.700394959564 |  0.993222999001  | 28036 | 5317 |
 | 0.00499999988824 | 0.689298476585 |  0.992794977886  | 28036 | 5317 |
 | 0.00600000005215 | 0.679706601467 |  0.992259951491  | 28036 | 5317 |
 | 0.00700000021607 | 0.669550498401 |  0.991831930375  | 28036 | 5317 |
 | 0.00800000037998 | 0.659206319353 |  0.991582251391  | 28036 | 5317 |
 | 0.00899999961257 

In [86]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [23]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [24]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'some': 1, 'it': 3, ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [25]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [26]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1, 'all': 1,""violet's"": 2, 'bring': ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'adoring': 1, 'find': 1,'month': 1, 'bright': 1, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2, 'discovered':1, 'existence.': 1, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2, ""don't"": 1,'(literally).so': 1, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1, 'all': 1,'reviews.': 2, 'just' ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2, 'both': 1,'month': 1, 'ears,': 1, ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5, 'the': 1,'all': 1, 'that': 2, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1, 'shape': 2,'mutt': 1, '""dog': 1, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4, 'chew': 1,'all': 1, 'perfect;': 1, ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1, 'all': 1,'out,': 1, 'over': 1, ...",0.999999985069


##Most positive reviews for the giraffe

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [28]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [27]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']


In [28]:
def awesome_count(x):
    if 'awesome' in x:
        return x['awesome']
    return 0


In [29]:
products[‘awesome’] = products[‘word_count’].apply(awesome_count)

SyntaxError: invalid syntax (<ipython-input-29-6fc22396d249>, line 1)

In [30]:
products['awesome'] = products['word_count'].apply(awesome_count)

RuntimeError: Runtime Exception. Unable to evaluate lambdas. lambda workers did not start

In [31]:
products['awesome'] = products['word_count'].apply(awesome_count)


name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1,0


In [33]:
products['awesome'].head()
    

dtype: int
Rows: 10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [34]:
products['awesome'].sum()


2002

In [35]:
list_of_lambdas = []
for word in selected_words:
    function = lambda x, word : 
    if word in x:
        return x[word]
    return 0
    list_of_lambdas.append(function)

SyntaxError: invalid syntax (<ipython-input-35-ca15d57ca352>, line 3)

In [36]:
list_of_lambdas = []
for word in selected_words:
    function = lambda x, word : if word in x return x[word] else return 0
    list_of_lambdas.append(function)

SyntaxError: invalid syntax (<ipython-input-36-e7198f447fe1>, line 3)

In [37]:
list_of_lambdas = []
for word in selected_words:
    function = lambda x, word : x[word] if word in x else 0
    list_of_lambdas.append(function)

In [38]:
len(list_of_lambdas
   )

11

In [39]:
for i in list_of_lambdas:
    products[selected_words[i]] = products['word_count'].apply(list_of_lambdas[i])

TypeError: list indices must be integers, not function

In [40]:
for i in len(list_of_lambdas):
    products[selected_words[i]] = products['word_count'].apply(list_of_lambdas[i])

TypeError: 'int' object is not iterable

In [41]:
for i in len(list_of_lambdas):
    print i


TypeError: 'int' object is not iterable

In [42]:
for i in range(len(list_of_lambdas)):
    print i


0
1
2
3
4
5
6
7
8
9
10


In [43]:
for i in range(len(list_of_lambdas)):
    products[selected_words[i]] = products['word_count'].apply(list_of_lambdas[i])

TypeError: <lambda>() takes exactly 2 arguments (1 given)

In [44]:
def great_count(x):
    if 'great' in x:
        return x['great']
    return 0

In [45]:
def fantastic_count(x):
    if 'fantastic' in x:
        return x['fantastic']
    return 0

In [46]:
def amazing_count(x):
    if 'amazing' in x:
        return x['amazing']
    return 0

In [47]:
def love_count(x):
    if 'love' in x:
        return x['love']
    return 0

In [48]:
def horrible_count(x):
    if 'horrible' in x:
        return x['horrible']
    return 0

In [49]:
def bad_count(x):
    if 'bad' in x:
        return x['bad']
    return 0

In [50]:
def terrible_count(x):
    if 'terrible' in x:
        return x['terrible']
    return 0

In [51]:
def awful_count(x):
    if 'awful' in x:
        return x['awful']
    return 0

In [52]:
def wow_count(x):
    if 'wow' in x:
        return x['wow']
    return 0

In [53]:
def hate_count(x):
    if 'hate' in x:
        return x['hate']
    return 0

In [54]:
products['great'] = products['word_count'].apply(great_count)

In [55]:
products['fantastic'] = products['word_count'].apply(fantastic_count)

In [56]:
products['amazing'] = products['word_count'].apply(amazing_count)

In [57]:
products['love'] = products['word_count'].apply(love_count)

In [58]:
products['horrible'] = products['word_count'].apply(horrible_count)

In [59]:
products['bad'] = products['word_count'].apply(bad_count)

In [60]:
products['terrible'] = products['word_count'].apply(terrible_count)

In [61]:
products['awful'] = products['word_count'].apply(awful_count)

In [62]:
products['wow'] = products['word_count'].apply(wow_count)

In [63]:
products['hate'] = products['word_count'].apply(hate_count)

In [64]:
products['great'].sum()

42420

In [65]:
products['fantastic'].sum()

873

In [66]:
products['amazing'].sum()

1305

In [67]:
products['love']/sum()

TypeError: sum expected at least 1 arguments, got 0

In [68]:
products['love'].sum()

40277

In [69]:
products['horrible'].sum()

659

In [70]:
products['bad'].sum()

3197

In [71]:
products['terrible'].sum()

673

In [72]:
products['awful'].sum()

345

In [73]:
products['wow'].sum()

131

In [74]:
products['hate'].sum()

1057

In [75]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

ToolkitError: The following columns were expected but are missing: ['great', 'love', 'hate', 'awesome', 'fantastic', 'terrible', 'bad', 'horrible', 'awful', 'amazing', 'wow']

In [76]:
train_data,test_data = products.random_split(.8, seed=0)


In [77]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.253864     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.410392     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.564343     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [78]:
selected_words_model[‘coefficients’]

SyntaxError: invalid syntax (<ipython-input-78-d8595bb092a0>, line 1)

In [79]:
selected_words_model['coefficients]

SyntaxError: EOL while scanning string literal (<ipython-input-79-3370d2204e36>, line 1)

In [100]:
selected_words_model['coefficients']

name,index,class,value
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
love,None,1,1.39989834302
horrible,None,1,-1.99651800559
bad,None,1,-0.985827369929
terrible,None,1,-2.09049998487
awful,None,1,-1.76469955631


In [83]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+----------------+-------+------+
 |    threshold     |        fpr        |      tpr       |   p   |  n   |
 +------------------+-------------------+----------------+-------+------+
 |       0.0        | 0.000188572506129 |      0.0       | 27969 | 5303 |
 | 0.0010000000475  |   0.999811427494  |      1.0       | 27969 | 5303 |
 | 0.00200000009499 |   0.999622854988  | 0.99996424613  | 27969 | 5303 |
 | 0.00300000002608 |   0.999622854988  | 0.99996424613  | 27969 | 5303 |
 | 0.00400000018999 |   0.999434282482  | 0.999928492259 | 27969 | 5303 |
 | 0.00499999988824 |   0.999434282482  | 0.999928492259 | 27969 | 5303 |
 | 0.00600000005215 |   0.999245709975  | 0.999892738389 | 27969 | 5303 |
 | 0.00700000021607 |   0.999245709975  | 0.999892738389 | 27969 | 5303 |
 | 0.00800000037998 |   0.999245709975  | 0.999892738389 | 27969 | 5303 |
 | 0.008

In [85]:
selected_words_model.show(view='Evaluation')

In [87]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [88]:
train_data,test_data = diaper_champ_reviews.random_split(.8, seed=0)

In [89]:
train_data,test_data = products.random_split(.8, seed=0)

In [90]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [91]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [92]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1, 'less': 1,""friend's"": 1, '(which': ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1, 'over': 1,'rweek': 1, 'sooo': 1, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2, 'just': 4,""don't"": 2, 'one,': 1, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1, 'able': 2,'over': 1, 'soon': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'son': 2, 'all': 1,'bags.': 1, 'son,': 1, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4, 'this': 3,'stink': 1, 'garbage' ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1, 'all': 2,'bags.': 1, 'feedback': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1, '-': 3,'both': 1, 'results': 1, ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'all': 1, 'humid': 1,'just': 1, 'less': 1, ...",1,0


In [93]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [94]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [95]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1,'contacted': 1, 'over': ...",1,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'instead': 1, 'all': 1,'already': 1, 'love': 3, ...",1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1, ""don't"": 1,'son.': 1, 'of,': 1, ...",1,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3, 'love': 1,'use.': 1, 'is': 2, ' ...",1,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2, 'bags.': 1,'garbage': 1, 'wastef ...",1,0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,"{'and': 3, 'love': 2,'being': 1, 'money': 1, ...",1,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'and': 1, 'old': 1,'extra': 1, 'is': 1, ...",1,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'and': 4, 'genies': 1,'all': 1, 'because': 1, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",1,0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'lysol': 1, 'all': 1,'just': 2, 'hand,': 1, ...",1,0


In [96]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [97]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [98]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [99]:
diaper_champ_reviews[0]

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 0,
 'hate': 0,
 'horrible': 0,
 'love': 0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9999999372669541,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor e